<a href="https://colab.research.google.com/github/ARJUN108-verma/LangGraph/blob/main/13_Long_term_memory_with_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Long term memory with SQLite:-

Set the OpenAI API Key as an Environment Variable

In [ ]:
!pip install mypy_ipython

In [ ]:
%load_ext dotenv
%dotenv
%load_ext mypy_ipython

Import Relevant Classes and Functions

In [ ]:
!pip install mypy_ipython

In [ ]:
!pip install langgraph

In [ ]:
!pip install langchain_openai

In [ ]:
!pip install langgraph.checkpoint.sqlite

In [ ]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage, RemoveMessage
from langgraph.checkpoint.memory import InMemorySaver
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

Define the Nodes

In [ ]:
class State(MessagesState):
    summary: str

In [ ]:
chat = ChatOpenAI(api_key="Enter API KEY here",
                  model = "gpt-4o",
                  seed = 365,
                  temperature = 0,
                  max_completion_tokens = 100)

In [ ]:
def ask_question(state: State) -> State:

    print(f"\n-------> ENTERING ask_question:")

    question = "What is your question?"
    print(question)

    return State(messages = [AIMessage(question), HumanMessage(input())])

In [ ]:
def chatbot(state: State) -> State:

    print(f"\n-------> ENTERING chatbot:")

    system_message = f'''
    Here's a quick summary of what's been discussed so far:
    {state.get("summary", "")}

    Keep this in mind as you answer the next question.
    '''

    response = chat.invoke([SystemMessage(system_message)] + state["messages"])
    response.pretty_print()

    return State(messages = [response])

In [ ]:
def summarize_messages(state: State) -> State:
    print(f"\n-------> ENTERING summarize_messages:")

    new_conversation = ""
    for i in state["messages"]:
        new_conversation += f"{i.type}: {i.content}\n\n"

    summary_instructions = f'''
Update the ongoing summary by incorporating the new lines of conversation below.
Build upon the previous summary rather than repeating it,
so that the result reflects the most recent context and developments.
Respond only with the summary.

Previous Summary:
{state.get("summary", "")}

New Conversation:
{new_conversation}
'''

    print(summary_instructions)

    summary = chat.invoke([HumanMessage(summary_instructions)])

    remove_messages = [RemoveMessage(id = i.id) for i in state["messages"][:]]

    return State(messages = remove_messages, summary = summary.content)

Define the Graph

In [ ]:
graph = StateGraph(State)

In [ ]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("summarize_messages", summarize_messages)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "summarize_messages")
graph.add_edge("summarize_messages", END)

In [ ]:
db_path = "C:/Users/Hristina/Desktop/LangGraph_DB/langgraph.db"
con = sqlite3.connect(database = db_path, check_same_thread = False)

In [ ]:
checkpointer = SqliteSaver(con)

In [ ]:
graph_compiled = graph.compile(checkpointer)

In [ ]:
graph_compiled

In [ ]:
config1 = {"configurable": {"thread_id": "1"}}

In [ ]:
graph_compiled.invoke(State(), config1)